# Asignement 3 = Stag Hunt model 
By Māra Učelniece 

- Response : there is not a specific data set we have to use, but just apply the evaluation of the structures to the EV's case that we are attempting to create policy interventions for. 

## Questions : 
- Ask Noah : Do we want to find a data set ? 

### Task : 
Evaluate how varying structures relate to policy and in which cases they might apply. 
   

In [3]:
# Imports 
    # Copied from exmaple notebook 
    
from mesa import Agent, Model
from mesa.time import SimultaneousActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
import networkx as nx
import numpy as np
import random
from typing import Iterable, List, Dict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed


# Can import the functions from the course_matterial 
    # Example : from file import function
        # from ./course_matterial/ev_core.py import ...
        
    # Example to import all functions
        # from ./course_matterial/ev_core.py import *


## 0. Implement a network agent based model

In [ ]:
# Generate the models 

# Def advanced in ev_core =  applicable cuz then we can adjsut the strategy if want to evaluate the rule

# But tghe class EVStagHuntModel(Model) also ... 

# Only till set_intial_adaptors the script seems usefull for now 



## 1. Baseline System Analysis

In [ ]:
# Evaluate trough plots 


## 2. Network Structure Analysis

Seems that the follwoing will be done with plots 
- speedofadoption,
- probability of reaching the high-adoption equilibrium,
- cluster formation,
- network-specific sensitivity to tipping.

In [4]:
# Mainly could use ev_plotting for this, but there is also some matterial in ev_experiments


## 3. Policy Intervention Designand Evaluation

In [5]:
# For this one the ev_experiments mainly perevelent: 
    # policy_subsidy_factory = Create a policy that temporarily boosts coordination payoffs
    # policy_infrastructure_boost_factory = Create a policy that injects infrastructure at a specific step
    
# However, these are only two interventions, and we wnat to test possibly more or others 
    # Can still be used for insparation / a basis to start from 
    